<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# The NISAR GSLC processor
#### Notebook kernel: plant

This notebook submits a query to identify an L1_L_RSLC product to be used to invoke the NISAR GSLC processor to produce an L2_L_GSLC.

## Setup configuration for interaction with on-demand NISAR SDS
In this section, we setup the necessary python libraries and API endpoints needed to interact with the **on-demand NISAR SDS**.

The `otello` python library provides high-level access to operations on the on-demand SDS. In particular for this demo, it provides us with access to and information about the job types registered on the SDS along with the capability to submit jobs, check for job statuses, and to query for products generated by the job.

The `pele` python library provides high-level read-only access to the dataset catalog managed by the on-demand SDS. In particular for this demo, it provides us with the capability to list the dataset types contained in the catalog, query for individual dataset granules of a particular dataset type, and to return metadata pertaining to an individual dataset granule.

In [ ]:
import os

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    !mkdir -p ./notebook_output/05a-Tunable_Parameters_GSLC
    os.chdir('notebook_output/05a-Tunable_Parameters_GSLC')
    
import json
import time
import re
import shutil
from glob import glob
from pprint import pprint
from datetime import datetime
from pip._internal.commands.show import search_packages_info
import logging
import matplotlib
logging.getLogger('matplotlib').setLevel(logging.WARNING)

import otello
from pele_client.client import PeleRequests

import plant
%matplotlib inline

print(f"PLAnT version: {plant.__version__}")

### Configure for interaction with the on-demand NISAR SDS

In [ ]:
# set the mozart IP
mozart_ip = "100.64.122.181"
#mozart_ip = input("Enter IP address of your mozart instance then press <Enter>: ")
print(f"Using mozart IP address {mozart_ip}.")

# instantiate otello and get mozart object to interact with SDS - you will be prompted for input values
# which will be defaulted on subsequent calls.
otello.client.initialize()
m = otello.mozart.Mozart()

# set pele URL
pele_url = f"https://{mozart_ip}/pele/api/v0.1"
print(f"Using pele URL {pele_url}.")

# instantiate PeleRequests object
pr = PeleRequests(pele_url, verify=False)

## Submitting a job for the NISAR GSLC processor
### Next let's get more information on the NISAR GSLC processor:

In [ ]:
print(m.get_job_types()["job-SCIFLO_GSLC:jdyoung-test"])

# get GSLC job type and initialize
job_type = "job-SCIFLO_GSLC:jdyoung-test"
gslc = m.get_job_types()[job_type]
gslc.initialize()

### Query NISAR SDS for an RSLC dataset
The input dataset for the NISAR GSLC processor is the NISAR RSLC product. Here we submit a geospatial query to the on-demand NISAR SDS for an input dataset. For an additional example on pele queries, see the _02-Datasets_ and _02-Datasets-geospatial_ notebooks.

In [ ]:
# query for all dataset IDs of the `L1_L_RSLC` dataset
r = pr.get(pele_url + '/pele/dataset/L1_L_RSLC/dataset_ids')
           
# expect 200
print("status code: {}".format(r.status_code))
res = r.json()
print(json.dumps(res, indent=2))
assert r.status_code == 200

datasets = res['dataset_ids']

search_poly=[[[-118,34],[-118,35],[-117,35],[-117,34],[-118,34]]]
search_start_time = '2008-02-18'
search_end_time = '2008-02-18T23:59:59Z'

# query for the dataset ID of the qualifying `L1_L_RSLC` dataset(s)
r = pr.post(pele_url + '/pele/dataset/L1_L_RSLC/dataset_ids', json = { 'polygon' : search_poly, 'start_time' : search_start_time, 'end_time' : search_end_time })
           
# expect 200
print("status code: {}".format(r.status_code))
res = r.json()
print(json.dumps(res, indent=2))
assert r.status_code == 200

datasets = res['dataset_ids']

In [ ]:
# use the first identified RSLC dataset from the above
r = pr.get(f"{pele_url}/pele/dataset/{datasets[0]}")
rslc_dataset = r.json()["result"]

In [ ]:
print(r.json())

### Prepare input for the on-demand GSLC processor

In [ ]:
# set dataset params
gslc.set_input_dataset(rslc_dataset)

### Submit the GSLC job to run in the on-demand NISAR SDS

In [ ]:
# submit job
job1 = gslc.submit_job(tag="gslc-test1-default-params")

### Let's wait for the job to complete generation of the standard GSLC product

Processing will take a few minutes to complete.

In [ ]:
# wait for job
job1.wait_for_completion()

### Get information about the generated GSLC product from the job
#### The generated GSLC product is stored in the cloud next to the on-demand NISAR SDS

In [ ]:
std_prod = job1.get_generated_products()
print(json.dumps(std_prod, indent=2, sort_keys=True))

### Download the generated standard GSLC product from the cloud into this notebook
#### Here we use the AWS CLI to download the generated dataset. Be sure that your AWS credentials are valid - refresh with aws-login in a terminal window if necessary.

Run the following cell if the job above succeeded.

In [ ]:
std_prod_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', std_prod[0]["urls"][-1]) # get s3 url
std_local_dir = os.path.basename(std_prod_url)

if os.path.isdir(std_local_dir): shutil.rmtree(std_local_dir)
print(std_local_dir)
!aws s3 sync $std_prod_url $std_local_dir

### Use PLAnT to visualize the output GSLC product
Here we list the contents of the generated dataset. The actual GSLC product is the HD5 file (*.h5).

In [ ]:
!ls -l $std_local_dir

#### Identify the GSLC product file

In [ ]:
import glob
gslc_file_names = glob.glob(os.path.join(std_local_dir, "*.h5"))
if(len(gslc_file_names) > 0):
    gslc_file_name = gslc_file_names[0]
    print(gslc_file_name)
else:
    print("No GSLC H5 file is present in {}".format(std_local_dir))

#### Read in the GSLC product

In [ ]:
import h5py

gslc_file = h5py.File(gslc_file_name, 'r')

#### Plot the HH polarization.

In [ ]:
# read in HH polarization from the GSLC
hh_polarization = gslc_file['science']['LSAR']['GSLC']['grids']['frequencyA']['HH']

# options
options = {}
# options['mute'] = True
# options['force'] = True

# display options
display_options = options.copy()

# display composition of HH polaraization.
plant.display(hh_polarization, title="test", **display_options)

<font size="1">This notebook is compatible with NISAR Jupyter Server Stack v1.7.1 and above</font>